In [1]:
%tensorflow_version 2.x

import tensorflow as tf
from tensorflow import keras
print("tf version {0} executing eagerly is {1}".format(tf.__version__, tf.executing_eagerly()))

import tensorflow_probability as tfp
print("tfp version {0}".format(tfp.__version__))

TensorFlow 2.x selected.
tf version 2.1.0 executing eagerly is True
tfp version 0.9.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = '/content/drive/My Drive/StoicNetData/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
device_name = tf.test.gpu_device_name()
print("Device name: \"{0}\"".format(device_name))
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Device name: "/device:GPU:0"
Found GPU at: /device:GPU:0
1 Physical GPUs, 1 Logical GPUs


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import cv2
import sys
import os
import time
import numpy as np
import glob
import PIL
import imageio
import matplotlib.pyplot as plt
from IPython import display

import warnings
warnings.simplefilter('error', UserWarning)
warnings.filterwarnings("error")

In [0]:
tf.random.set_seed(98475651423)

In [0]:
IMG_HEIGHT = 112
IMG_WIDTH = 112

keys_to_features = {
    'image_neutral': tf.io.FixedLenFeature([], tf.string),
    'image_expressive': tf.io.FixedLenFeature([], tf.string),
    'image_other': tf.io.FixedLenFeature([], tf.string)
}

def parser(record):
    parsed = tf.io.parse_single_example(record, keys_to_features)
    
    imageNeutral = tf.io.decode_raw(parsed["image_neutral"], tf.uint8)
    imageExpressive = tf.io.decode_raw(parsed["image_expressive"], tf.uint8)
    imageOther = tf.io.decode_raw(parsed["image_other"], tf.uint8)
    
    imageNeutral = tf.cast(imageNeutral, tf.float32)
    imageExpressive = tf.cast(imageExpressive, tf.float32)
    imageOther = tf.cast(imageOther, tf.float32)
    
    imageNeutral = tf.reshape(imageNeutral, shape=[224,224,1])
    imageExpressive = tf.reshape(imageExpressive, shape=[224,224,1])
    imageOther = tf.reshape(imageOther, shape=[224,224,1])

    imageNeutral = tf.image.resize(imageNeutral, size=[IMG_HEIGHT,IMG_WIDTH])
    imageExpressive = tf.image.resize(imageExpressive, size=[IMG_HEIGHT,IMG_WIDTH])
    imageOther = tf.image.resize(imageOther, size=[IMG_HEIGHT,IMG_WIDTH])

    imageNeutral /= (255/2)
    imageExpressive /= (255/2)
    imageOther /= (255/2)
    imageNeutral -= 1
    imageExpressive -= 1
    imageOther -= 1

    return {"imageNeutral":imageNeutral, "imageExpressive":imageExpressive, "imageOther":imageOther}

In [7]:
BUFFER_SIZE = 20000
BATCH_SIZE = 32

DB_PATH = BASE_DIR
raw_train = tf.data.TFRecordDataset(DB_PATH + "train.tfrecords")
raw_test = tf.data.TFRecordDataset(DB_PATH + "test.tfrecords")

parsed_train = raw_train.map(parser).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
parsed_test = raw_test.map(parser).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(parsed_train)

<BatchDataset shapes: {imageNeutral: (None, 112, 112, 1), imageExpressive: (None, 112, 112, 1), imageOther: (None, 112, 112, 1)}, types: {imageNeutral: tf.float32, imageExpressive: tf.float32, imageOther: tf.float32}>


In [0]:
class TripletLoss(tf.keras.losses.Loss):
    def __init__(self, alpha):
        super(TripletLoss, self).__init__()
        self.alpha = alpha

    def call(self, y_true, y_pred):
        anc = y_pred[0]; pos = y_pred[1]; neg = y_pred[2]
        # distance between the anchor and the positive
        #pos_dist = tf.reduce_sum(tf.square(tf.subtract(anc, pos)), axis=-1)
        pos_dist = tf.reduce_sum(tf.subtract(anc, pos), axis=-1)    
        # distance between the anchor and the negative
        #neg_dist = tf.reduce_sum(tf.square(tf.subtract(anc, neg)), axis=-1)
        neg_dist = tf.reduce_sum(tf.subtract(anc, neg), axis=-1)
        
        return tf.math.maximum(tf.add(tf.subtract(pos_dist, neg_dist), self.alpha), 0.0)


alpha = tf.convert_to_tensor(0.2, dtype=tf.float32)
triplet_loss = TripletLoss(alpha)

In [0]:
class Recognizer(tf.keras.Model):
    def __init__(self, latent_dim):
        super(Recognizer, self).__init__()
        self.encoder = tf.keras.models.Sequential(layers=[
            tf.keras.layers.InputLayer(input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
 
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2,2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2,2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),

            tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=(2,2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),

            tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=(2,2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(latent_dim)
        ], name="encoder")

        self.comparer = tf.keras.models.Sequential(layers=[
            tf.keras.layers.InputLayer(input_shape=(latent_dim)),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ], name="comparer")

    
    @tf.function
    def recognize(self, a, b):
        a_encoding = recognizer(a)
        b_encoding = recognizer(b)

        diffs = tf.abs(a_encoding - b_encoding)

        prediction = comparer(diffs)
        return prediction



latent_dim = 100
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00006, beta_1=0.5)
recognizer = Recognizer(latent_dim)


In [0]:
checkpoint_dir = BASE_DIR + 'training_checkpoints/recog_training_1/cp.ckpt'
checkpoint = tf.train.Checkpoint(optimizer=optimizer,recognizer=recognizer)

In [0]:
def plot_loss(ls):
    fig = plt.figure()
    plt.plot(ls, 'r')
    plt.title("Loss")
    plt.tight_layout()
    plt.show()

In [0]:
def compute_loss(rec, x_neutral, x_express, x_other):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    pos_preds = rec.recognize(x_express, x_neutral)
    neg_preds = rec.recognize(x_express, x_other)
    
    pos_labels = tf.ones_like(pos_preds)
    pos_loss = cross_entropy(pos_labels, pos_preds)

    neg_labels = tf.zeros_like(neg_preds)
    neg_loss = cross_entropy(neg_labels, neg_preds)

    return pos_loss + neg_loss

In [25]:
epochs = 50

ls = []
loss = -1
start_time = end_time = 0
for epoch in range(1, epochs+1):
    print('Epoch {0}, Test set loss {1}, time elapsed for current epoch {2:.1f}'.format(
        epoch, loss, end_time - start_time))
    # Test
    loss = tf.keras.metrics.Mean()
    for image_features in parsed_test:
        l = compute_loss(recognizer, neutral_raw, expressive_raw, other_raw)
        loss(l)
    l = loss.result()
    ls = ls + [l]

    plot_loss(ls)


    # Train
    start_time = time.time()
    batchCounter = 0
    for image_features in parsed_train:
        neutral_raw = image_features['imageNeutral']
        assert not np.any(np.isnan(neutral_raw))
        expressive_raw = image_features['imageExpressive']
        assert not np.any(np.isnan(expressive_raw))
        other_raw = image_features['imageOther']
        assert not np.any(np.isnan(other_raw))

        with tf.GradientTape() as tape:
            loss = compute_loss(recognizer, neutral_raw, expressive_raw, other_raw)
        
        gradients = tape.gradient(loss, recognizer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, recognizer.trainable_variables))

        batchCounter += 1
        if batchCounter % 10 == 0:
            print(".", end="")
    end_time = time.time()

Epoch 1, Test set loss -1, time elapsed for current epoch 0.0


NameError: ignored